In [5]:
import pandas as pd
import tkinter as tk
from tkinter import ttk  # Import ttk module for Treeview
from tkinter import StringVar

df = None  # Define df at a higher scope
tree = None  # Define tree at a higher scope

def carregar_arquivo():
    global df, tree
    url = entrada_url.get()

    # Tentar ler o arquivo com a codificação 'utf-8'
    try:
        df = pd.read_csv(url, sep=',', on_bad_lines='skip', dtype=str, encoding='utf-8')
    except Exception as e:
        print(f"Erro ao carregar o arquivo: {str(e)}")
        return

    # Get the user-specified column names or indices
    colunas_selecionadas = entrada_colunas.get().split(',')

    # Filter columns based on user input
    if colunas_selecionadas[0] == '':
        colunas_selecionadas = df.columns.tolist()
    else:
        colunas_selecionadas = [col.strip() for col in colunas_selecionadas]

    # Create Combobox widgets for filtering by column names
    filter_comboboxes = {}
    for col in colunas_selecionadas:
        filtro = StringVar()
        filtro.set("Todas")  # Define o valor padrão como "Todas"
        combobox = ttk.Combobox(frame_filtros, textvariable=filtro)
        combobox['values'] = ["Todas"] + list(df[col].unique())
        combobox.grid(row=0, column=colunas_selecionadas.index(col), sticky="w")
        filter_comboboxes[col] = filtro

    # Create a filter button
    botao_filtrar = tk.Button(frame_filtros, text="Filtrar", command=filtar_tabela)
    botao_filtrar.grid(row=1, column=0, columnspan=len(colunas_selecionadas), pady=5)

    # Create a new Treeview widget with columns and headings
    columns = colunas_selecionadas

    # Create a frame for the Treeview with scrollbars
    tree_frame = ttk.Frame(root)
    tree_frame.grid(row=3, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")
    tree_frame.grid_rowconfigure(0, weight=1)
    tree_frame.grid_columnconfigure(0, weight=1)

    canvas = tk.Canvas(tree_frame)
    canvas.grid(row=0, column=0, sticky="nsew")

    vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=canvas.yview)
    vsb.grid(row=0, column=1, sticky="ns")
    canvas.configure(yscrollcommand=vsb.set)

    tree = ttk.Treeview(canvas, columns=columns, show="headings")
    tree.grid(row=0, column=0, sticky="nsew")

    hsb = ttk.Scrollbar(tree_frame, orient="horizontal", command=canvas.xview)
    hsb.grid(row=1, column=0, sticky="ew")
    canvas.configure(xscrollcommand=hsb.set)

    canvas.create_window((0, 0), window=tree, anchor="nw")
    canvas.update_idletasks()
    canvas.configure(scrollregion=canvas.bbox("all"))

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    mostrar_tabela(tree)

def mostrar_tabela(tree):
    if df is not None:
        for i in tree.get_children():
            tree.delete(i)
        for index, row in df.iterrows():
            values = [row[col] for col in df.columns]
            tree.insert("", "end", values=values)

def filtar_tabela():
    if df is not None:
        selected_filters = {col: filtro.get() for col, filtro in filter_comboboxes.items()}
        mask = pd.Series([True] * len(df))
        for col, value in selected_filters.items():
            if value != "Todas" and col in df.columns:
                mask &= (df[col] == value)
        for i in tree.get_children():
            tree.delete(i)
        for index, row in df[mask].iterrows():
            values = [row[col] for col in df.columns]
            tree.insert("", "end", values=values)

root = tk.Tk()
root.title("Leitor de CSV Online")

frame_url = tk.Frame(root)
frame_url.grid(row=0, column=0, padx=10, pady=5, sticky="w")
label_url = tk.Label(frame_url, text="URL do CSV:")
label_url.grid(row=0, column=0)
entrada_url = tk.Entry(frame_url)
entrada_url.grid(row=0, column=1)

frame_colunas = tk.Frame(root)
frame_colunas.grid(row=1, column=0, padx=10, pady=5, sticky="w")
label_colunas = tk.Label(frame_colunas, text="Colunas (separadas por vírgula):")
label_colunas.grid(row=0, column=0)
entrada_colunas = tk.Entry(frame_colunas)
entrada_colunas.grid(row=0, column=1)

frame_filtros = tk.Frame(root)
frame_filtros.grid(row=2, column=0, columnspan=2, padx=10, pady=5, sticky="w")

botao_carregar = tk.Button(root, text="Carregar Arquivo", command=carregar_arquivo)
botao_carregar.grid(row=3, column=0, columnspan=2, pady=10)

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Eduarda\AppData\Local\Temp\ipykernel_13688\3852741485.py", line 86, in filtar_tabela
    selected_filters = {col: filtro.get() for col, filtro in filter_comboboxes.items()}
                                                             ^^^^^^^^^^^^^^^^^
NameError: name 'filter_comboboxes' is not defined
